**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2021*

----

# Trabajo práctico entregable - Parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

### Importación de librerías

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import missingno as msno

import seaborn as sns
sns.set_context('talk')

pd.set_option('display.max_columns', None)

In [2]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

'5.2.2'

In [3]:
# To update plotly, uncomment and run the following line:
# !pip install plotly --upgrade

In [4]:
melb_df = pd.read_csv('https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


---

## Ejercicio 1: 

1. Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

2. Investigar la distribución de las variables del conjunto de datos y seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad.
    1. Justificar cada columna no seleccionada.
    2. Para las variables categóricas seleccionadas, agrupe o combine las categorías poco frecuentes para asegurar que todos los grupos tengan un número mínimo de registros.

3. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 
    1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
    2. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
    3. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.


In [5]:
# data source:
# https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]
airbnb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv',
    usecols=interesting_cols,low_memory=False)

KeyboardInterrupt: 

In [ ]:
airbnb_df[:3]

### Análisis del Datasets y Tipos de Datos

Como primer paso, daremos un vistazo general a los tipos de datos dentro del dataset **melb_df**

In [ ]:
melb_df.info()

Antes de comenzar, vemos que la columna **Date** está como un String, por lo que la haremos una nueva variable Date transformandola en formato de fecha para lograr un mejor análisis sobre los datos.

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df.Date)
print('Cantidad de fechas distintas:', melb_df['Date'].nunique())
melb_df['Date'].head()

In [ ]:
print('Fecha desde:',melb_df['Date'].min())
print('Fecha Hasta:',melb_df['Date'].max())

Analizamos el código postal (**Postcode**)

In [ ]:
print('Cantidad de valores nulos:',melb_df.Postcode.isna().sum())

In [ ]:
melb_df.Postcode.dtype

In [ ]:
melb_df.Postcode.unique()

Como el código postal no es una variable cuantitativa, por lo que para asegurar que sea categórica/cualitativa, la transformamos en tipo de datos String

In [ ]:
melb_df.Postcode = melb_df.Postcode.astype(str)
melb_df.Postcode = melb_df.Postcode.str[0:4]
melb_df.Postcode.dtype

In [ ]:
melb_df.Postcode.value_counts()

In [ ]:
print('Códigos postales desde:', min(melb_df.Postcode.values))
print('Códigos postales desde:', max(melb_df.Postcode.values))

Observaremos la falta de datos o los valores nulos dentro de la tabla, lo cual implican que debamos decidir que acciones se deben tomar para estos casos. 

In [ ]:
melb_df.isna().sum()

De igual manera, realizamos la búsqueda de los valores en 0 dentro del dataset, para analizar la relevancia de la información presentada.

In [ ]:
cols = melb_df[melb_df == 0].count(axis=0)
cols[cols > 0]

*  Como punto importante, observamos que dirección, precio (price) y suburbio no tienen valores nulos, lo cual es beneficioso para el análisis de esta consigna.
* Podemos observar las columnas que poseen valores en 0, las cuales pueden considerarse para un futuro descarte, ya que no aportan información significativa para el análisis para esta consigna. La decisión de descartarlos o mantenerlos, se decidirá mientras se avance con el desarrollo para no producir desvión o introducir sesgos que no correspondan. 

---

### Actividad 1

**CONSIGNA**

> Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

Decribiremos a la columna **Price** (Precio) para ver las características de la misma.

In [ ]:
melb_df[['Price']].describe().round(2)

In [ ]:
print('Mediana:',melb_df[['Price']].median())

Observamos que se registraron 13.580 propiedades en dicho dataset, con un precio de valor medio de 1.075.684 USD, una propiedad con un precio mínimo de 85.000 USD y una propiedad con precio máximo de 9M USD.

Las conclusiones rápidas que podemos notar son: Con una media de 1.075.684 USD y un máximo y mínimo ya dichos, claramente hay outliers en valores altos, ya que en su mayoría, la concentración de valores se da en el **cuartil 3**. Esto se da ya que **la media es mayor a la mediana**. 

Demostraremos en una visualización de caja o boxenplot, lo concluido en el paso anterior: 

In [ ]:
plt.figure(figsize=(16, 7))
sns.boxenplot(data=melb_df, x="Price", color='salmon')
plt.title("Grafico de Caja de la distribución de la V.A. Price (Sin Limpiar Outliers)", fontsize=15)
plt.show()

El el gráfico, visualizamos que la variable presenta valores menos concentrados en cuanto más altos estos se hacen, y se puede intuir, que por encima de 4M USD, estos se hacen más atípicos.

Para analizar esta hipótesis, visualizaremos las propiedades que superan el monto de 4M USD para comparar las caracteristicas con las que cuentan.

In [ ]:
melb_df[melb_df['Price'] >= 4000000].head(70)

In [ ]:
melb_df[melb_df['Price'] >= 4000000].describe()

Observamos que las propiedades por encima de 4M tienen características que podrían justificar el precio de la propiedades, como por ejemplo la cantidad de habitaciones, o el área en donde se encuentra, como así también el área construida.

Luego de este análisis, optamos por sacar los outliers superiores por el método que utilizan los diagramas de cajas para este análisis.
El método consta de  2 partes:
1. Sacar el rango intercuartil: Q3 - Q1
2. Obtener los bigotes, tanto superior como inferior. Para el inferior se le resta al Q1 el rango intercuartil por 1.5, y para el superior es el Q3 más el rango intercuartil  multiplicado por 1.5.

El valor 1.5 es un valor dado para valores atípicos leves, pero pueden utilizarse valores superiores si es de preferencia.

Adjuntamos   [link de Wikipedia](https://es.wikipedia.org/wiki/Valor_at%C3%ADpico#:~:text=%2C%20o%20rango%20intercuart%C3%ADlico.,esa%20distancia%20(at%C3%ADpico%20extremo). en donde se puede obtener más detalle sobre el método utilizado.



In [ ]:
def range_interquantile(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    # RI = Rango_Intercuartil
    RI = Q3 - Q1
    # BI = Bigote Inferior
    BI = Q1 - 1.5 * RI
    # BS = Bigote Superior
    BS = Q3 + 1.5 * RI
    return RI, BI, BS

In [ ]:
RI, BI, BS = range_interquantile(melb_df, 'Price')

print('Rango Intercuartil:',RI)
print('Bigote Inferior:', BI)
print('Bigote Superior:',BS)

Aquí obtuvimos 3 valores, nuestro rango intercuartil (Q3-Q1), el bigote superior el cual nos limitará nuestro dataset con sus valores superiores, y el bigote inferior que nos limitará el dataset con los valores mínimos.

Notesé que el bigote inferior es negativo, por lo cual queda totalmente descartado para nuestros valores, porque como ya comprobamos anteriormente, no existen datos negativos para precio.

Procederemos entonces a sacar los outliers en un nuevo dataset:

In [ ]:
outliers = len(melb_df[melb_df['Price'] > BS])
print('Datos considerados como Outliers:', outliers)

In [ ]:
melb_df[melb_df['Price'] > BS].describe().round(2)

In [ ]:
def remove_outliers(data, column):
    RI, BI, BS = range_interquantile(data, column)
    data = data[data[column] > BI]
    data = data[data[column] < BS]
    return  data

In [ ]:
melb_df_2 = remove_outliers(melb_df, 'Price')
melb_df_2

Como podemos observar, la cantidad de propiedades que quedan es de 12.956.

Vamos a visualizar una vez más el diagrama boxenplot con este nuevo dataset para corroborar las diferencias.

In [ ]:
fig, axes = plt.subplots(2, 1,figsize=(14,16))

sns.boxenplot(data=melb_df, x="Price", color='salmon', ax=axes[0])
axes[0].title.set_text('Dataset Original - V.A. Price - Con Outliers')
sns.boxenplot(data=melb_df_2, x="Price", color='green', ax=axes[1])
axes[1].title.set_text('Dataset Nuevo - V.A. Price - Sin Outliers')

plt.show()

Observamos, que la concentración de los valores es mas uniforme y concentrados. Donde los valores extremos superiores fueron quitados, dejando así un dataset más homogeneos y significativo para realizar el análisis. 

Volvemos a describir la columna PRICE de este nuevo dataset para visualizar sus caracteristicas: 

In [ ]:
melb_df_2[['Price']].describe().round(2)

In [ ]:
print('Mediana:',melb_df_2[['Price']].median())

---

#### Conclusión 1.1

Podes concluir, que quitar oultiers nos benefició en: 

*   El valor mínimo para el precio queda en 85.000 USD y el nuevo máximo es 2.345.000 USD. 
* Nuestro nuevo valor de media es 981.359 USD.
* La mediana no sufrió tanto movimiento como si fue el de la media.
* La desviación estandar bajó por 200.000 lo cual nos dice que los datos están menos dispersos.

---

### Actividad 2

**CONSIGNA**

> Investigar la distribución de las variables del conjunto de datos y seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad.

    1. Justificar cada columna no seleccionada.
    2. Para las variables categóricas seleccionadas, agrupe o combine las categorías poco frecuentes para asegurar que todos los grupos tengan un número mínimo de registros.

Visualizamos una vez más el nuevo dataset transformado, el cual se trabajo la quita de los outliers: 

In [ ]:
melb_df_2.head()

Con la observación de los datos y la definición de las columanos, podemos considerar cuales serán descartadas y cuales son de mayor relevancias y ser tomadas en el análisis.

Para corroborar que la elección es la correcta, se analizaran las caracteristicas de las mismas. 

In [ ]:
cols_2b_discarded = ['Method', 'SellerG', 'Bedroom2', 'Propertycount']

In [ ]:
melb_df_2[cols_2b_discarded].info()

In [ ]:
melb_df_2[cols_2b_discarded].describe()

Observamos los valores nulos de estas columnas a descartar.

In [ ]:
plt.figure(figsize=(13, 6))
msno.bar(melb_df_2[cols_2b_discarded],figsize=(13, 6), fontsize=12, color='lightblue')
plt.show()

Analizamos la presencia de valores 0 sobre las posibles variables a descartar.

In [ ]:
meld_disc = melb_df_2[cols_2b_discarded]
cols = meld_disc[meld_disc == 0].count(axis=0)
cols

Observamos que dentro de las columnas que queremos descartar, Bedroom2 es la única que posee valores en 0, lo cual tiene cierta lógica según la definición dada por quién ofrece el dataset.

Llegado a este punto nos surge la siguiente pregunta: **¿Qué ocurre con las columnas que no queremos descartar?**

Analizaremos si estas tienen valores en 0 y/o nulos.

In [ ]:
meld_no_disc = melb_df_2[['Suburb', 'Address', 'Rooms', 'Type', 'Price','Date', 'Distance', 'Bathroom', 'Car','CouncilArea', 'Postcode', 'Lattitude', 'Longtitude',
                     'Landsize', 'BuildingArea', 'YearBuilt','Regionname']]
cols = meld_no_disc[meld_no_disc == 0].count(axis=0)
cols

In [ ]:
msno.bar(meld_no_disc,figsize=(13, 6), fontsize=12, color='lightblue')
plt.show()

In [ ]:
msno.matrix(meld_no_disc,figsize=(13, 6), fontsize=12, color=[0.5,0,0])

Vemos que las columnas con mayor cantidad de registros en 0 es **Landsize** y en cuanto a valores nulos, **BuildingArea** y **YearBuilt** son las más afectadas.

Por el momento, estas no serán descartadas, ya que puede que sus datos sean salvados y/o imputados.

Siguiendo con el análisis de las columnas descartadas, vemos los valores de la columna **SellerG** para ver qué contiene específicamente.

In [ ]:
print(melb_df_2.SellerG.unique())
print(len(melb_df_2.SellerG.unique()))

Podemos concluir rápidamente, que una propiedad puede ser vendida por mas de un Gerente de Ventas y si el precio varía dependiendo el mismo, no creemos que influya a grandes rasgos.

Daremos un análisis a la columna Bedroom2, la cual deseamos descartar, porque consideramos poco relevante para el análisis el hecho de tener o no una columna que nos indica "Habitación de huespedes" cuando ya se especifica la cantidad de habitaciones de una propiedad. 

In [ ]:
pd.crosstab(melb_df_2.Bedroom2,melb_df_2.Rooms)

#### Columnas Descartadas
* Method: Otra de las columnas poco relevantes, porque nos dice cómo fue vendida.
* SellerG: Variable ya analizada con anterioridad, y que justificamos diciendo que el vendedor puede ser cualquier persona, que no consideramos que influya demasiado en el valor de la propiedad.
* Bedroom2: Ya visto anteriormente, consideramos que anunciando la cantidad de habitaciones es suficiente para el análisis.
* Propertycount: Descartada porque no consideramos que influya la cantidad de propiedades en el precio final de la propiedad a analizar.

---

#### Columnas Seleccionadas

In [ ]:
relevant_columns = ['Suburb', 'Address', 'Rooms', 'Type', 'Price','Date', 'Distance', 'Bathroom', 'Car', 'CouncilArea', 'Postcode', 'Lattitude', 'Longtitude',
                    'Landsize', 'BuildingArea', 'YearBuilt','Regionname']

Creamos un nuevo dataset para el análisis requerido:

In [ ]:
melb_df_RC = melb_df_2[relevant_columns]
melb_df_RC

Visualizamos la cantidad de valores únicos dentro de las variables seleccionadas.

In [ ]:
melb_df_RC.nunique()

In [ ]:
melb_df_RC.describe().round(2)

Analizaremos las columnas que podemos agrupar.

Tomaremos en primera instancia, las variables categorícas **Regionname** y **YearBuilt**.

---

##### Agrupamiento **Regionname**

Observamos los valores únicos para **Regionname** y la cantidad de valores que tienen.

In [ ]:
melb_df_RC[['Regionname']].groupby(by=['Regionname']).size()

Observando las columnas y la cantidad de valores de cada una, vamos a hacer clustering en 4 regiones:
* South (Sur)
* North (Norte)
* East (Este)
* West (Oeste)

Creamos un diccionario de las regiones con su nueva definición

In [ ]:
dic_region = {'Eastern Metropolitan': 'East', 'Eastern Victoria': 'East', 'Northern Metropolitan': 'North', 
              'Northern Victoria': 'North', 'South-Eastern Metropolitan': 'South', 'Southern Metropolitan': 'South',
              'Western Metropolitan': 'West', 'Western Victoria': 'West'}

Ya contando con el diccionario de las regiones, se reemplazará la región correspondiente para cada propiedad en el nuevo dataset determinado para el analisis.

In [ ]:
melb_df_RC = melb_df_RC.replace({'Regionname': dic_region})
melb_df_RC[['Regionname']].groupby(by=['Regionname']).size()

---

##### Agrupamiento YearBuilt en AgeRange

Ahora trataremos con los valores de años de contrucción. Primero analizamos qué contiene nuestro datasets

In [ ]:
melb_df_RC[['YearBuilt']].groupby(by=['YearBuilt']).size()

Analizaremos este dato con un gráfico, para comparar la frecuencia de los años de construcción de las propiedades del dataset. 

In [ ]:
fig = plt.figure(figsize=(16, 8))
sns.histplot(data=melb_df_RC[['YearBuilt']], x="YearBuilt", kde=True, bins=100, color='red', stat= 'density')
plt.title('Distribución de la columna YearBuilt',fontsize=16)
plt.xlabel('YearBuilt (Año de construcción)')
plt.show()

Como vemos, hay valores desde antes de 1900. Analizaremos la concentración de datos.

In [ ]:
plt.figure(figsize=(16, 7))
sns.boxenplot(data=melb_df_RC[['YearBuilt']], x="YearBuilt", color='salmon')
plt.title("Gráfico para analizar concentración de valores", fontsize=15)
plt.show()

Observamos un outlier o un caso muy atípico que se presenta en una propiedad construida en 1196. Este pudo o no ser un dato mal tipificado, pero por el momento no tomamos decisiones de cambio sobre el registro.

Miraremos la concentración dentro de las propiedades construidas por encima de 1800.

In [ ]:
plt.figure(figsize=(16, 7))
sns.boxenplot(data=melb_df_RC[melb_df_RC.YearBuilt >= 1800], x="YearBuilt", color='salmon')
plt.title("Gráfico para analizar concentración de valores", fontsize=15)
plt.show()

In [ ]:
melb_df_RC[melb_df_RC.YearBuilt >= 1800].describe().round(2)

Como observamos, dentro de la columna **YearBuilt** existe el registro con valor 2018, generando un dato ruidoso con respecto a la columna **Date**, en donde el valor máximo de fecha es del año 2017.

Analicemos lo que pasa en este registro.

In [ ]:
melb_df_RC[melb_df_RC.YearBuilt >= 2018]

Luego de este análisis, podemos deducir que hay propiedades que se venden, y posterio se construyen, porque en este registro de la columna **YearBuilt**, el año de construcción es posterior al año de la venta.

In [ ]:
melb_df_RC[pd.DatetimeIndex(melb_df_RC['Date']).year < melb_df_RC.YearBuilt]

Podemos concluir, que para este caso, existen varias propiedades (5), con el año de construcción mayor al año de venta.

Crearemos una nueva columna, informando el rango de antiguedad correspondiente a cada propiedad según su año de construcción y fecha de venta.

In [ ]:
melb_df_RC.head(10)

In [ ]:
melb_df_RC['Antiquity'] = pd.DatetimeIndex(melb_df_RC['Date']).year - melb_df_RC['YearBuilt']
melb_df_RC.loc[melb_df_RC['Antiquity'] < 0, 'Antiquity'] = 0
print('Cantidad de valores únicos:', melb_df_RC['Antiquity'].nunique())
print(melb_df_RC[['Antiquity']].info())

Al ver que tenemos 144 valores, distintos para antigüedad, procedemos a crear la columna de rango de antiguedad.

In [ ]:
def year_range(value):
        if value >= 70:
            return '[>70)'
        else:
            low_limit = value//5
            agerange = "["+str(int(low_limit*5))+"-"+str(int(low_limit*5+5))+")"
            return agerange        

In [ ]:
#melb_df_RC['AgeRange'] = melb_df_RC['Antiquity'].apply(year_range)
melb_df_RC['AgeRange'] = melb_df_RC[melb_df_RC['Antiquity'].isna()==False]['Antiquity'].apply(year_range)
melb_df_RC.loc[melb_df_RC['AgeRange'].isnull(), 'AgeRange'] = 'S/D'
melb_df_RC.AgeRange.unique()

##### Distribución

A partir de este punto, comenzaremos a realizar las distribuciones por precio de las propiedades respecto a las variables escogidas.

In [ ]:
PriceToAgeRange = melb_df_RC[['AgeRange','Price']].sort_values(by=['AgeRange'])
fig = plt.figure(figsize=(18, 8))
ax = sns.barplot(x='AgeRange', y='Price', data=PriceToAgeRange, color='#35A7FF')
plt.xticks(rotation=45)
plt.title('Distribución de Precios Medios respecto a la Antigüedad de la Propiedad',fontsize=22)
plt.ylabel("Media del Precio de la Propiedad")
plt.xlabel("Rango Antigüedad")
plt.ticklabel_format(style='plain', axis='y')
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = sns.barplot(x='Regionname', y='Price', data=melb_df_RC)
plt.show()

Borrado de las columnas que nos ayudaron a agrupar.

In [ ]:
melb_df_RC.drop(columns=['Antiquity'],inplace=True)
melb_df_RC

---

### Actividad 3

Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico. 

1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
2. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
3. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

In [ ]:
airbnb_df.head(10)

#### Análisis y Curación Extra

In [ ]:
airbnb_df.info()

In [ ]:
airbnb_df.describe()

In [ ]:
airbnb_df.isna().sum()

##### Columnas **zipcode**

In [71]:
airbnb_df.zipcode.str.len().unique()

array([ 4., nan,  5.,  8.,  9., 10.,  3.])

El código postal debe tener un largo de 4 dígitos, por lo que listaremos los códigos postales del DataFrame

In [72]:
airbnb_df.zipcode.unique()

array(['3105', '3057', '3182', '3071', '3183', '3806', '3073', '3002',
       '3166', '3199', '3121', '3000', '3187', '3101', '3065', '3188',
       '3068', '3195', '3070', '3207', '3205', '3181', '3135', '3052',
       '3004', '3013', '3160', '3777', '3079', '3184', '3149', '3206',
       '3123', '3053', '3161', '3094', '3056', '3046', '3051', '3066',
       '3006', '3141', '3130', '3023', '3159', '3162', '3030', '3167',
       '3186', '3067', '3015', '3143', '3103', '3910', '3980', '3029',
       '3084', '3185', '3144', '3145', '3031', '3104', '3012', '3072',
       '3191', '3058', '3074', '3775', '3163', nan, '30122', '3147',
       '3789', '3175', '3099', '3018', '3011', '3078', '3788', '3107',
       '3042', '3040', '3178', '3134', '3089', '3142', '3911', '3168',
       '3008', '3129', '3146', '3190', '3088', '3193', '3131', '3204',
       '3032', '3109', '3797', '3782', '3003', 'VIC 3161', '3113', '3081',
       '3158', '3122', '3136', '3140', '3076', '3153', '3977', '3044',
    

In [85]:
airbnb_df.zipcode = airbnb_df.zipcode.str.extract('(\d+)').rename(columns={0:'zipcode'}).zipcode.str[0:4]
airbnb_df.zipcode = airbnb_df.zipcode.astype(str)
airbnb_df.head()

,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0
3,"Comfortable, relaxed house, a home away from ...","This is a great neighbourhood – it is quiet, y...","Thornbury, VIC, Australia",Thornbury,Darebin,Thornbury,VIC,3071,-37.758971,144.989228,50,250.0,920.0
4,We offer comfortable accommodation in Inner Me...,NaN,"St Kilda East, VIC, Australia",St Kilda East,Port Phillip,St Kilda East,VIC,3183,-37.864530,144.992238,69,400.0,1900.0


In [87]:
airbnb_df.zipcode

0        3105
1        3057
2        3182
3        3071
4        3183
         ... 
22890    3053
22891    3338
22892    3150
22893    3006
22894    3053
Name: zipcode, Length: 22895, dtype: object

In [86]:
print('Código postal mínimo:', min(airbnb_df.zipcode.values))
print('Código postal máximo:', max(airbnb_df.zipcode.values))

Código postal mínimo: 2010
Código postal máximo: nan


In [ ]:
Si vamos 

##### Columna **price**

Distribución de la columnas **price**

In [ ]:
plt.figure(figsize=(16, 6))
sns.boxenplot(data=airbnb_df, x="price", color='salmon')
plt.title("Distribución de la V.A. price", fontsize=15)
plt.show()

In [ ]:
RIair, BIair, BSair = range_interquantile(airbnb_df, 'price')

print('Rango Intercuartil:',RIair)
print('Bigote Inferior:', BIair)
print('Bigote Superior:',BSair)

In [ ]:
airbnb_df.price.describe()

Procederemos entonces a sacar los outliers de airbnb_df en un nuevo dataset:

In [ ]:
outliers = len(airbnb_df[airbnb_df['price'] > BSair])
print('Datos considerados como Outliers:', outliers)

In [ ]:
airbnb_df[airbnb_df['price'] > BSair].describe().round(2)

In [ ]:
airbnb_df_2 = remove_outliers(airbnb_df, 'price')
airbnb_df_2

In [ ]:
plt.figure(figsize=(16, 6))
sns.boxenplot(data=airbnb_df_2, x="price", color='salmon')
plt.title("Distribución de la V.A. price (Sin Outliers)", fontsize=15)
plt.show()

##### Análisis de Nulos

In [ ]:
msno.bar(airbnb_df_2,figsize=(14, 5), fontsize=12, color='lightblue')
plt.title('Análisis de Nulos', fontsize=18)
plt.show()

In [ ]:
msno.matrix(airbnb_df_2,figsize=(14,7), fontsize=12, color=[0.5,0,0])
plt.title('Análisis de Nulos', fontsize=18)
plt.show()

Observando las columnas **weekly_price** y **monthly_price**, se puede visualizar una gran cantidad de nulos. Se descartarán ya que no aportan información al modelo actual.

Podemos ver que las columnas **neighborhood_overview** y **neighborhood**, que son las siguientes con mayor cantidad de nulos.

In [ ]:
airbnb_df_2[['neighborhood_overview','neighborhood']].nunique()

Se observa que:
* La columna **neighborhood** tiene 16004/21448 de datos, los cuales 59 son únicos. Se analizarán y se tomará una decisión sobre si se debe descartar.
* La columna **neighborhood_overview** tiene 13492/21448 de datos, los cuales 11297 son únicos. Se toma la decisión que es una variable 

In [ ]:
airbnb_df_2.neighborhood.value_counts()

In [ ]:
print('Porcentaje de filas nulas:', round((airbnb_df_2.neighborhood.isna().sum() / len(airbnb_df_2)) * 100,2))

La columna **neighborhood** será descartada por la cantidad de valores en nulo y no tener mucho conocimiento de dominio, ya que si hubiera manera de imputarlo y que luego nos sirviera por cada código postal, se la dejaría para enriquecer los datos.

##### Columnas posibles a descartar

Buscando en internet y comprendiendo nuestros conjuntos de datos determinamos lo siguiente:
* El país es Australia, el cual posee varios estados, y dentro, ciudades. Estamos estudiando el caso de Melbourne, del estado de Victoria.
* La ciudad que nos interesa es Melbourne y no otra, pero, vimos que Melbourne es una metrópolis. Veremos el análisis más adelante.
* La columna **suburb** es importante, ya que marca las diferentes partes o subdivisiones de Melbourne.
* El estado es Virginia, pero analizaremos el datasets para ver si se descarta.

In [ ]:
airbnb_df_2[['description','street','state']]

In [ ]:
airbnb_df_2[['description','street','state']].nunique()

In [ ]:
airbnb_df_2['state'].value_counts()

In [ ]:
airbnb_df_2.city.value_counts()

Antes de descartar la columna **city**, observaremos la relación entre código postal y ciudad

In [ ]:
airbnb_df_2[airbnb_df_2['city'] != 'Melbourne'].city.nunique()

In [ ]:
zipcodes_mb = airbnb_df_2[airbnb_df_2['city'] == 'Melbourne'].zipcode.unique()
citys = airbnb_df_2[airbnb_df_2['city'] != 'Melbourne'].city.unique()
abnb_df_s_mb = airbnb_df_2[airbnb_df_2['city'] != 'Melbourne'][['city','zipcode']].drop_duplicates()

aux_df = pd.DataFrame()
count = 0
ci = []
zipcodes_r = []
for c in citys:
    for z in zipcodes_mb:
        if len(abnb_df_s_mb[(abnb_df_s_mb['city'] == c) & (abnb_df_s_mb['zipcode'] == z)])  > 0:
            count += 1
            ci.append(c)
            zipcodes_r.append(z)

aux_df['City'] = ci
aux_df['Zipcode'] = zipcodes_r

aux_df.sort_values(by=['City','Zipcode']).reset_index().drop(columns=['index'])

Buscando los códigos postales de Melbourne dentro de los códigos postales de las otras ciudades, encontramos que existen coincidencias.

Se buscaron en internet estas ciudades, y encontramos que son parte de Melbourne, por lo que quitar la columna **city** tal vez sea una equivocación. Lo que haremos, será dejarla y dejar la moda a partir del código postal.

Posibles columnas a descartar:
* La columna **description** es una variable categórica con muchos valores únicos y creemos que no nos aporta información necesaria para nuestro análisis. Se descartará.
* La columna **street** es una variable categórica con muchos valores únicos con nombre de la calle, estado y país. Estamos tratando con Melbour, una Ciudad del Estado de Victoria en Australia. Como no creemos que aporte información necesaria, decidimos descartarla.
* La columna **state** es una variable categórica con varios estados dentro del datasets. El único que priorizamos es "Victoria", por lo que no es una variable que va a cambiar y no aportará información. Se descartará.
* La columna **city** es una variable categórica que no descartaremos, y usaremos la moda como valor principal a partir del Código postal.

#### Selección de Columnas AirBnb

Resumen de selección:
* Se descartarán: 'description', 'neighborhood_overview', 'street', 'neighborhood', 'state', 'weekly_price', 'monthly_price'
* Columnas seleccionadas: 'zipcode', 'city', 'suburb', 'latitude', 'longitude', 'price'

In [ ]:
relevant_columns = ['zipcode', 'city', 'suburb', 'latitude', 'longitude', 'price']

Procesos de cada columna:
* **zipcode**: Variable Categórica principal en el caso, se procederá con una agrupación.
* **city**: se tomará la moda.
* **suburb**: se tomará la moda.
* **latitude**: se tomará la media.
* **longitude**: se tomará la media.
* **price**: se tomará la media.

In [ ]:
airbnb_df_final = airbnb_df_2.groupby(by=['zipcode']) \
                            .agg(airbnb_city = ('city',pd.Series.mode), 
                                 airbnb_suburb = ('suburb',pd.Series.mode), 
                                 airbnb_latitude = ('latitude','mean'), 
                                 airbnb_longitude = ('longitude','mean'), 
                                 airbnb_price = ('price','mean')).reset_index()
airbnb_df_final.airbnb_price = airbnb_df_final.airbnb_price.round(2)
airbnb_df_final

In [ ]:
airbnb_df_final.shape

#### Unión de Datasets por Código Postal

Podemos observar que podemos usar el código postal para realizar una unión entre datasets.

In [ ]:
intersection = np.intersect1d(
    airbnb_df_final.zipcode.values, melb_df_RC.Postcode.values, assume_unique=False)
print("Valores únicos de zipcodes en Airbnb:", len(airbnb_df_final.zipcode.unique()))
print("Valores únicos de zipcodes en melb_df_RC:", len(melb_df_RC.Postcode.unique()))
print("Códigos postales en común:", len(intersection))

In [ ]:
print('Records in Sales df with corresponding zipcode form Airbnb df',
      melb_df_RC.Postcode.isin(intersection).sum() / len(melb_df_RC))
print('Records in Airbnb df with corresponding zipcode form Sales df',
      airbnb_df_final.zipcode.isin(intersection).sum() / len(airbnb_df_final))

In [ ]:
airbnb_df.zipcode.str.extract('(\d+)').rename(columns={0:'zipcode'}).zipcode.str[0:4].unique()

¿Un posible ordenamiento sería el siguiente...? Entre paréntesis anotamos lo que consideramos sería una traducción más familiar para nosotros
state (provincia) > Regionname (zona) > CouncilArea (municipio) > city (ciudad) > Suburb (barrio) > neighborhood (¿barrio de vuelta?) > street (calle)

In [ ]:
airbnb_df.zipcode.str[:4].unique()

In [ ]:
airbnb_df.zipcode.unique()

In [ ]:
airbnb_df.zipcode.str.len().unique()

## Ejercicio 2: Imputación

1. Imputar los valores faltantes de la columna CouncilArea en base a la información presente en Suburb. (Si en el punto anterior no seleccionó ninguna de estas variables, agréguelas ahora el conjunto de datos).




In [ ]:
melb_df_RC.CouncilArea.value_counts()

In [ ]:
melb_df_RC2=melb_df_RC.copy(deep=True)
melb_df_RC2

In [ ]:
melb_df_RC['CouncilArea'].isnull().sum(axis = 0)

In [ ]:
from sklearn_pandas import CategoricalImputer

imputer = CategoricalImputer()
melb_df_RC2['CouncilArea']=imputer.fit_transform(melb_df_RC2['CouncilArea'])


In [ ]:
melb_df_RC2['CouncilArea'].value_counts

In [ ]:
#melb_df_RC['CouncilArea']=melb_df_RC2['CouncilArea']

Tras realizar la imputación sobre el campo **CouncilArea**, podemos observar que los valores faltantes fueron reemplazados por **Moreland** que es la categoria más frecuente que se visualiza.             

2. Imputar los valores faltantes de la(s) columna(s) que se agregaron a partir del conjunto de datos de AirBnB.


In [ ]:
# Juan se me hizo lio aca para saber cuales son las columnas de AirBnB. ¿Esta en el df melb_de_RD? ¿O debo imputar sobre las columnas del df airbnb_df??

## Ejercicio 3

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.